In [2]:
pwd


'/home/wsuser/work'

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# Training Datagen
train_datagen = ImageDataGenerator(rescale=1/255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)
# Testing Datagen
test_datagen = ImageDataGenerator(rescale=1/255)


In [7]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='b6NEXYI14B7pNWBp8ejwJbjmnEdSymULPYV2N4AzlvTl',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'workshopdeployments-donotdelete-pr-teqshqaihxveje'
object_key = 'Data_Collection.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [8]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [9]:
pwd

'/home/wsuser/work'

In [10]:
# Training Dataset
x_train=train_datagen.flow_from_directory('/home/wsuser/work/Data Collection/training_set',target_size=(64,64), class_mode='categorical',batch_size=900)
# Testing Dataset
x_test=test_datagen.flow_from_directory('/home/wsuser/work/Data Collection/test_set',target_size=(64,64), class_mode='categorical',batch_size=900)


Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


In [11]:
print("Len x-train : ", len(x_train))
print("Len x-test : ", len(x_test))

Len x-train :  18
Len x-test :  3


In [12]:
# The Class Indices in Training Dataset
x_train.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

In [13]:
# Importing Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense

In [14]:
# Creating Model
model=Sequential()

In [15]:
# Adding Layers
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64,64,3)))

In [16]:
# Adding Pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [17]:
# Adding Flattem layer
model.add(Flatten())

In [18]:
# Adding Dense Layers
model.add(Dense(300,activation='relu'))
model.add(Dense(150,activation='relu'))
model.add(Dense(9,activation='softmax'))

In [19]:
# Compiling the Model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
# Fitting the Model Generator
model.fit_generator(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))

/tmp/wsuser/ipykernel_164/1042518445.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))


Epoch 1/10
18/18 [==============================] - 41s 2s/step - loss: 1.0942 - accuracy: 0.6411 - val_loss: 0.4000 - val_accuracy: 0.9071
Epoch 2/10
18/18 [==============================] - 40s 2s/step - loss: 0.2196 - accuracy: 0.9345 - val_loss: 0.2756 - val_accuracy: 0.9373
Epoch 3/10
18/18 [==============================] - 42s 2s/step - loss: 0.0900 - accuracy: 0.9752 - val_loss: 0.2237 - val_accuracy: 0.9649
Epoch 4/10
18/18 [==============================] - 40s 2s/step - loss: 0.0439 - accuracy: 0.9889 - val_loss: 0.2141 - val_accuracy: 0.9738
Epoch 5/10
18/18 [==============================] - 40s 2s/step - loss: 0.0281 - accuracy: 0.9925 - val_loss: 0.2350 - val_accuracy: 0.9769
Epoch 6/10
18/18 [==============================] - 40s 2s/step - loss: 0.0160 - accuracy: 0.9972 - val_loss: 0.2553 - val_accuracy: 0.9742
Epoch 7/10
18/18 [==============================] - 41s 2s/step - loss: 0.0130 - accuracy: 0.9975 - val_loss: 0.2544 - val_accuracy: 0.9773
Epoch 8/10
18/18 [==

In [21]:
model.save('IBM_asl_model-36394.h5')